In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.tiingo_extractor import TiingoExtractor

In [2]:
market = Market()
market.connect()
dates = market.retrieve_date_range()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
stuff = pd.to_datetime(dates["date"])

In [4]:
start = (stuff.max() + timedelta(days=1)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)

2021-12-01 2021-12-02


In [5]:
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        data = TiingoExtractor.extract(ticker,start,end)
        data["ticker"] = ticker
        market.store("prices",data)
    except Exception as e:
        print(str(e))
market.disconnect()

 16%|████████████▌                                                                 | 81/505 [01:00<05:24,  1.31it/s]

project_market prices documents must be a non-empty list


 61%|██████████████████████████████████████████████▋                              | 306/505 [03:52<02:26,  1.36it/s]

project_market prices documents must be a non-empty list


 99%|████████████████████████████████████████████████████████████████████████████▌| 502/505 [06:15<00:02,  1.29it/s]

...

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


100%|████████████████████████████████████████████████████████████████████████████▊| 504/505 [06:17<00:00,  1.31it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


100%|█████████████████████████████████████████████████████████████████████████████| 505/505 [06:18<00:00,  1.34it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


In [6]:
data

In [7]:
analysis = []
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_data(ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        periods = ["year","quarter","month","week"]
        ep = test.tail(1)["adjClose"].item()
        # periods = ["year"]
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(str(e))
market.disconnect()

 13%|██████████                                                                    | 65/505 [02:02<13:43,  1.87s/it]

'date'


 16%|████████████▏                                                                 | 79/505 [02:29<13:17,  1.87s/it]

'date'


 99%|████████████████████████████████████████████████████████████████████████████▌| 502/505 [15:56<00:05,  1.90s/it]

...

'date'


100%|████████████████████████████████████████████████████████████████████████████▊| 504/505 [15:59<00:01,  1.88s/it]

'date'


100%|█████████████████████████████████████████████████████████████████████████████| 505/505 [16:01<00:00,  1.90s/it]

'date'


In [8]:
a = pd.DataFrame(analysis).merge(sp5.rename(columns={"Symbol":"ticker"})[["ticker","Security","GICS Sector"]],on="ticker",how="left")
a.to_csv("analysis.csv")

In [9]:
period = "day"
a.sort_values(f"{period}_gain",ascending=False).head(50)

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker,Security,GICS Sector
469,-0.106481,0.136679,0.0,0.078323,0.096609,VRTX,Vertex Pharmaceuticals,Health Care
346,0.337547,0.129402,0.0,0.023256,0.034298,NVR,NVR,Consumer Discretionary
282,0.473435,0.158698,0.0,0.012308,0.033508,LEN,Lennar,Consumer Discretionary
234,0.542029,0.298896,0.0,0.014751,0.033447,HPQ,HP,Information Technology
45,0.755302,0.174553,0.0,0.002976,0.030437,AMAT,Applied Materials,Information Technology
488,-0.231819,-0.209834,0.0,0.004332,0.025917,WU,Western Union,Information Technology
74,-0.083641,-0.076626,0.0,0.005673,0.024613,BMY,Bristol-Myers Squibb,Health Care
78,0.365318,0.000000,0.0,0.008613,0.022518,COG,Cabot Oil & Gas,Energy
195,1.298122,0.382768,0.0,-0.004575,0.020323,F,Ford,Consumer Discretionary
277,0.460768,0.217296,0.0,0.019088,0.020122,LRCX,Lam Research,Information Technology


In [10]:
a.groupby("GICS Sector").mean().sort_values("day_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain
GICS Sector,,,,,
Utilities,0.083603,0.012087,0.0,-0.029934,-0.000087
Health Care,0.140244,-0.037484,0.0,-0.028652,-0.005844
Consumer Staples,0.041301,-0.009864,0.0,-0.034044,-0.006545
Energy,0.530746,-0.015522,0.0,-0.038233,-0.012530
Materials,0.214363,0.016762,0.0,-0.045344,-0.013795
Information Technology,0.218575,0.031698,0.0,-0.033972,-0.014121
Financials,0.305340,-0.023146,0.0,-0.041977,-0.014365
Real Estate,0.389713,0.021546,0.0,-0.039727,-0.016103
Industrials,0.209636,0.003425,0.0,-0.043254,-0.016493
